In [83]:
import pandas as pd

In [84]:
train_data = pd.read_csv('train_vidya3.csv')

In [85]:
test_data = pd.read_csv('test_vidya3.csv')

In [86]:
mode = train_data["DOB"].mode()
train_data["DOB"].where(pd.isnull(train_data["DOB"])==False,mode[0],inplace=True)

In [87]:
train_data['DOB'] = train_data['DOB'].apply(lambda x: int(str(x)[-2:]))

In [89]:
train_data['Lead_Creation_Date'] = train_data['Lead_Creation_Date'].apply(lambda x: int(str(x)[3:5]))*100+train_data['Lead_Creation_Date'].apply(lambda x: int(str(x)[:2]))

In [61]:
mode = train_data["City_Category"].mode()
train_data["City_Category"].fillna(mode[0],inplace = True)

In [88]:
sum(pd.isna(train_data["DOB"])==True)

0

In [62]:
mode = train_data["Employer_Category1"].mode()
train_data["Employer_Category1"].fillna(mode[0],inplace = True)

In [63]:
train_data["Existing_EMI"].fillna(545436.500000,inplace = True)

In [64]:
mode = train_data["Primary_Bank_Type"].mode()
train_data["Primary_Bank_Type"].fillna(mode[0],inplace = True)

In [65]:
mode = train_data["Customer_Existing_Primary_Bank_Code"].mode()
train_data["Customer_Existing_Primary_Bank_Code"].fillna(mode[0],inplace = True)

In [66]:
train_data['Customer_Existing_Primary_Bank_Code'] = train_data['Customer_Existing_Primary_Bank_Code'].apply(lambda x: int(str(x)[-3:]))

In [67]:
train_data["Source_Category"].where(train_data["Source_Category"]!='D','E',inplace=True)
train_data["Source_Category"].where(train_data["Source_Category"]!='F','E',inplace=True)
train_data["Source_Category"].where(train_data["Source_Category"]!='A','E',inplace=True)

In [68]:
from sklearn.base import TransformerMixin,BaseEstimator
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self,attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [69]:
num_attribs = ["DOB","Lead_Creation_Date","Employer_Category2","Monthly_Income","Existing_EMI","Var1","Customer_Existing_Primary_Bank_Code"]

In [70]:
from sklearn.preprocessing import LabelBinarizer
class LabelBinarizer_new(TransformerMixin, BaseEstimator):
    def fit(self, X, y = 0):
        self.encoder = None
        return self
    def transform(self, X, y = 0):
        if(self.encoder is None):
            print("Initializing encoder")
            self.encoder = LabelBinarizer();
            result = self.encoder.fit_transform(X)
        else:
            result = self.encoder.transform(X)
        return result; 

In [71]:
from sklearn.pipeline import Pipeline

In [72]:
num_pipeline = Pipeline([("selector",DataFrameSelector(num_attribs))])

In [73]:
cat_pipeline1 = Pipeline([("selector",DataFrameSelector("Gender")),("label_binarizer",LabelBinarizer_new())])

In [74]:
cat_pipeline2 = Pipeline([("selector",DataFrameSelector("City_Category")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline3 = Pipeline([("selector",DataFrameSelector("Employer_Category1")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline4 = Pipeline([("selector",DataFrameSelector("Contacted")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline5 = Pipeline([("selector",DataFrameSelector("Source_Category")),("label_binarizer",LabelBinarizer_new())])
cat_pipeline6 = Pipeline([("selector",DataFrameSelector("Primary_Bank_Type")),("label_binarizer",LabelBinarizer_new())])

In [75]:
from sklearn.pipeline import FeatureUnion

In [76]:
full_pipeline = FeatureUnion(transformer_list = [("num_pipeline",num_pipeline),("cat_pipeline1",cat_pipeline1),("cat_pipeline2",cat_pipeline2),("cat_pipeline3",cat_pipeline3),("cat_pipeline4",cat_pipeline4),("cat_pipeline5",cat_pipeline5),("cat_pipeline6",cat_pipeline6)])

In [77]:
X_train = full_pipeline.fit_transform(train_data)

Initializing encoder
Initializing encoder
Initializing encoder
Initializing encoder
Initializing encoder
Initializing encoder


In [78]:
X_train.shape
y_train = train_data["Approved"].copy()

In [79]:
from sklearn.ensemble import RandomForestClassifier

In [80]:
forest_clf = RandomForestClassifier(n_estimators=100)

In [81]:
from sklearn.model_selection import cross_val_score

In [82]:
cross_val_score(forest_clf,X_train,y_train,cv=3,scoring="roc_auc")

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').